# API https://open-meteo.com/en/docs/historical-weather-api#api_form

In [ ]:
import requests
import pandas as pd

## Lat and lon

In [ ]:
coord = pd.read_excel("df_pref_coord.xlsx")

In [ ]:
coord.head()

## API request

In [ ]:
resp = []
lat = coord["lat"]
lon = coord["lon"]

for l in range(len(coord)):
    response_weather = requests.get(f"https://archive-api.open-meteo.com/v1/archive?latitude={lat[l]}3&longitude={lon[l]}&start_date=2013-01-01&end_date=2023-01-08&daily=weathercode,temperature_2m_max,temperature_2m_min,sunrise,sunset,shortwave_radiation_sum,precipitation_sum,windspeed_10m_max&timezone=Europe%2FBerlin")
    
    if response_weather.status_code == 200:
            resp.append(response_weather.json())
    else: 
            print("Request to {} failed".format(l))

In [ ]:
resp

In [ ]:
weather_info = []

for n in range(len(coord)):
    weather_data = [resp[n]["latitude"], 
            resp[n]["longitude"],
            resp[n]["daily"]["time"],
            resp[n]["daily"]["weathercode"],
            resp[n]["daily"]["temperature_2m_max"],
            resp[n]["daily"]["temperature_2m_min"],
            resp[n]["daily"]["shortwave_radiation_sum"],
            resp[n]["daily"]["precipitation_sum"],
            resp[n]["daily"]["windspeed_10m_max"]]
    weather_info.append(weather_data)
weather_info

## to dataframe

In [ ]:
weather_df = pd.DataFrame(data=weather_info)

In [ ]:
weather_df

In [ ]:
weather_df["cities"] = coord["cities"]
weather_df["code_region"] = coord["code_region"]
weather_df["region_nom"] = coord["region_nom"]
weather_df["depart_num"] = coord["depart_num"]
weather_df["depart_nom"] = coord["depart_nom"]
weather_df.head()

In [ ]:
weather = weather_df.explode(column = [2, 3, 4, 5, 6, 7, 8])

In [ ]:
weather = weather.rename(columns = {0:"lat", 1:"lon", 2: "time", 3:"weathercode", 4: "temp_max", 5: "temp_min", 6: "hours_of_sun", 7:"precipitation", 8:"windspeed" })
weather = weather.reset_index(drop = True)
weather.head()

In [ ]:
# Basic stats
print("Number of rows : {}".format(weather.shape[0]))
print()

print("Display of dataset: ")
display(weather.head())
print()

print("Basics statistics: ")
data_desc = weather.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*weather.isnull().sum()/weather.shape[0])

print("count different datatypes: ")
weather.dtypes.value_counts()


In [ ]:
weather[weather["weathercode"].isnull()]

- missing data from Jan 4th to jan 8th 2023

## preparing for merge with other elements

### Aggregation at Region level

In [ ]:
agg_dict = {"temp_max": "mean", "temp_min": "mean", "hours_of_sun":"mean", "precipitation":"mean", "windspeed": "mean", "region_nom": "max" }
weather_def = weather.groupby(by = ["code_region","time"]).agg(agg_dict)


In [ ]:
weather_def.head()

In [ ]:
weather_def.shape

In [ ]:
weather_def = weather_def.reset_index()
weather_def.head()

In [ ]:
weather_def.columns

Adding a field for merge with other file

In [ ]:
weather_def["key"] = weather_def["time"].astype(str) + weather_def["code_region"].astype(str)

In [ ]:
weather_def.head()

In [ ]:
weather_def.to_csv("weather.csv")

In [ ]:
weather_def.shape